<a href="https://colab.research.google.com/github/dtabuena/Patch_Ephys/blob/main/input_resistance_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import scipy as sci
#  scipy.signal. import butter,filtfilt

In [ ]:
def input_resistance_analyzer(abf, dVm_limits = [-30, 10],to_plot=False,figopt={'type':'jpg','dpi':300}):
    '''Calulates the series of delta Vs and delta Is and fits with a line to find the resistance.'''

    results={}


    stim_currents = []
    ss_voltage = []
    is_base, is_stim = protocol_baseline_and_stim(abf)
    for s in abf.sweepList:
        abf.setSweep(s)
        delta_v, _, _ = sweep_VIR(abf.sweepY, abf.sampleRate, is_stim = is_stim)
        delta_I, _, _    = sweep_VIR(abf.sweepC, abf.sampleRate, is_stim = is_stim) # repurpose but for command current
        stim_currents.append( delta_I)
        ss_voltage.append(delta_v)

    stim_currents = np.array(stim_currents)
    ss_voltage = np.array(ss_voltage)
    vm_in_range = [(dV>=dVm_limits[0] and dV<=dVm_limits[1]) for dV in ss_voltage]

    dI_to_fit = stim_currents[vm_in_range]
    dV_to_fit = ss_voltage[vm_in_range]

    inputR_fit = {}
    slope, intercept , r_value, p_value, std_err = sci.stats.linregress(dI_to_fit, dV_to_fit)
    Rsqr = r_value**2

    dV_hat = dI_to_fit*slope+intercept

    fit_err = (dV_hat - dV_to_fit)**2
    std = np.std(fit_err)
    z_err = fit_err/std
    outlier = z_err > 3

    if np.any(outlier):
        dI_to_fit = dI_to_fit[np.logical_not(outlier)]
        dV_to_fit = dV_to_fit[np.logical_not(outlier)]
        slope, intercept , r_value, p_value, std_err = sci.stats.linregress(dI_to_fit, dV_to_fit)
        Rsqr = r_value**2
        dV_hat = dI_to_fit*slope+intercept

    if to_plot:
        fig,ax = plt.subplots(1,1,figsize=(1.5,1.5))
        ax.scatter(stim_currents, ss_voltage)
        ax.scatter(dI_to_fit, dV_to_fit,color='red')
        ax.plot( dI_to_fit, dV_hat )
        ax.set_xlabel('Current (pA)')
        ax.set_ylabel('Delta V (mV)')
        # plt.show()
        try:    os.makedirs('Saved_Figs/Input_Resitance/')
        except:     None
        fig.savefig( 'Saved_Figs/Input_Resitance/Input_Resitance_' + abf.abfID +'.'+figopt['type'],dpi=figopt['dpi'])


    results['Input_Resistance_MO'] = slope*1E9*1E-6   # to ohms to megaohms
    results['Rin_Rsqr'] = Rsqr

    return results

def sweep_VIR(trace,rate,is_stim = None, window_t=0.100):
    '''Takes a trace and calulates the steady state delta V from a stimulus in Current Clamp'''
    if any(is_stim == None):
        is_stim = [True for i in trace]
    base_v = trace[:np.where(is_stim==True)[0][0]]
    cutoff = 5
    nyq = rate/2
    normal_cutoff = cutoff / nyq
    b, a = sci.signal.butter(3, normal_cutoff, btype='low')
    filtered_step_v = sci.signal.filtfilt(b, a, trace[is_stim])
    window_wid = int(window_t*rate)
    med_base_v = np.median(base_v[-window_wid:-1])
    med_stim_v = np.median(filtered_step_v[-window_wid:-1])
    delta_v = med_stim_v - med_base_v
    return delta_v, med_base_v, med_stim_v



# fn = 'my_ephys_data/2022_08_16/2022x08x16_RNF182_E4KI_F_P255_s004_c007_CA3xPOS_0007.abf'
# abf = abf_or_name(fn)
# input_resistance_analyzer(abf,to_plot=True)